In [3]:
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader

from langchain.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import OpenAI

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate,MessagesPlaceholder, PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

from langchain.vectorstores import Chroma
from langchain.document_loaders import WebBaseLoader

import json

# Resume Short Listing using Vector Storage and Similarity Search

The motive of this chain is to shortlist a set of resumes that match the closest with a given job description. This section effectively uses the capabilities of Document Embeddings to retrieve relevant documents. 

In [5]:
# Load a sample data set of resumes
directory = "/Users/chandu/Documents/hiring-assistant/data/"
documents = []

for dirpath, dirnames, filename in os.walk(directory):
    if dirnames != []:
        for folder in dirnames:
            
            # if folder == "ENGINEERING":
            #     print(folder)
            #     continue
            category = dirpath + folder
            count = 0
            for f in os.listdir(category):
                resume_file = category + "/" + f
                loader = PyPDFLoader(resume_file)
                documents.extend(loader.load())
                count += 1
                if count == 5:
                    break
                # print(category + "/"  +f)


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)


In [6]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(chunks, embeddings)

/Users/chandu/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
# Load a Job Description from a given URL. 
url = "https://www.amazon.jobs/en/jobs/2573804/software-development-engineer"
# url = "https://careers.compassgroupcareers.com/executive-chef-iii-fps-san-diego-ca/job/28276702"
loader = WebBaseLoader(url)
data = loader.load()

model = ChatOpenAI(model="gpt-4o")
system_template = """Enclosed in *** is a webpage of a job posting. Extract all the necessary information about the job and organize the information in the following categories. 
1) Title
2) Skills
3) Minimum Qualifications
4) Preferred Qualifications

Here is an example of how you can extract the required information from a given job description. 

====EXAMPLE START====
== START JOB DESCRIPTION==
Summary
Posted: Jun 25, 2024
Weekly Hours: 40
Role Number:200556628
Be a significant part of the Hardware Test Engineering Core Infrastructure team that is at the center of the Apple Factory’s Development Process. This team is responsible for application development to improve delivery, management, and scalability of factory test platforms to factories located worldwide. We are looking for a curious, hardworking, self-motivated software developer to work on several key systems as a member of our collaborative team.
Description
We are currently seeking an individual with strong backend software skills and a willingness to work with both C++ and Swift. 
You will work with a team of extraordinary engineers to develop and deploy mass-production friendly solutions for all Apple products. This would include architecting, developing, deploying and triaging software used in the factory during manufacture of new products. Responsibilities would include triaging and diagnosing issues with cross-functional teams from diverse engineering backgrounds during new product bringup, as well as working with external vendors on defining better solutions and processes.
Minimum Qualifications

    Backend software development experience
    Extensive expertise with Objective-C, C++, or Swift
    Proficient with UNIX/Linux or macOS
    Experience with scripting languages (Bash, Python, Perl)
    Excellent knowledge of software testing methodologies & practices
    Excellent knowledge of source code management concepts and systems (Git)
    Effective communication and an ability to collaborate with those around them

Preferred Qualifications

    4+ years of backend software development experience in Unix or Linux environment with expertise in C++
    BS degree in Computer Science, Computer Engineering or equivalent practical experience.

Pay & Benefits

    At Apple, base pay is one part of our total compensation package and is determined within a range. This provides the opportunity to progress as you grow and develop within a role. The base pay range for this role is between $135,400 and $250,600, and your base pay will depend on your skills, qualifications, experience, and location.

    Apple employees also have the opportunity to become an Apple shareholder through participation in Apple’s discretionary employee stock programs. Apple employees are eligible for discretionary restricted stock unit awards, and can purchase Apple stock at a discount if voluntarily participating in Apple’s Employee Stock Purchase Plan. You’ll also receive benefits including: Comprehensive medical and dental coverage, retirement benefits, a range of discounted products and free services, and for formal education related to advancing your career at Apple, reimbursement for certain educational expenses — including tuition. Additionally, this role might be eligible for discretionary bonuses or commission payments as well as relocation. Learn more about Apple Benefits.

    Note: Apple benefit, compensation and employee stock programs are subject to eligibility requirements and other terms of the applicable plan or program.

    Apple is an equal opportunity employer that is committed to inclusion and diversity. We take affirmative action to ensure equal opportunity for all applicants without regard to race, color, religion, sex, sexual orientation, gender identity, national origin, disability, Veteran status, or other legally protected characteristics. Learn more about your EEO rights as an applicant.
==END JOB DESCRIPTION==

    Title: Software Engineer, Software and Services, Apple
    Skills: Objective-C, C++, Swift, UNIX/Linux, macOS, Bash, Python, Perl
    Minimum Qualifications: Backend software development experience,Experience with scripting languages, Excellent knowledge of software testing methodologies & practices, Excellent knowledge of source code management concepts and systems, Effective communication and an ability to collaborate with those around them
    Preferred Qualifications: 4+ years of backend software development experience in Unix or Linux environment with expertise in C++, BS degree in Computer Science, Computer Engineering or equivalent practical experience.

====EXAMPLE END====

***
{web_page}
***

Just give your final summary, no need to give any additional text. 

"""


system_message_prompt = PromptTemplate.from_template(system_template)
final_prompt = system_message_prompt.format(web_page=data[0].page_content)

job_description = model.invoke(final_prompt)

print(job_description.content)



Title: Software Development Engineer

Skills: Java, HTML/CSS, Python, SQL, C, C++, data structures, algorithms, arrays, hash tables, graphs, problem solving, root-cause analysis, advanced English proficiency (B2+ or higher)

Minimum Qualifications: Completed university bachelor’s degree in Computer Science, Computer Engineering, System Engineering or related field in the last 12 months (or currently enrolled in last year of bachelor's study program), experience with programming languages (Java, HTML/CSS, Python, SQL, C, C++), proficiency in data structures, algorithms, arrays, hash tables and graphs, problem solving, and root-cause analysis, advanced English proficiency (B2+ or higher)

Preferred Qualifications: Previous technical internship experience with programming/coding


In [9]:
def find_relevant_resumes(job_description, top_k=20):
    results = vectorstore.similarity_search(job_description, k=top_k)
    return results

# job_description = "We are looking for a Chef with experience in multi cultured cuisines."
relevant_resumes = find_relevant_resumes(job_description.content)

visited_docs = set()

final_list = []

for doc in relevant_resumes:
    if doc.metadata['source'] in visited_docs:
        continue
    print(f"Relevant document: {doc.metadata['source']}")
    final_list.append(doc.metadata['source'])
    visited_docs.add(doc.metadata['source'])
    print("---")


Relevant document: /Users/chandu/Documents/hiring-assistant/data/BPO/41152404.pdf
---
Relevant document: /Users/chandu/Documents/hiring-assistant/data/CONSULTANT/22351830.pdf
---
Relevant document: /Users/chandu/Documents/hiring-assistant/data/ENGINEERING/28762662.pdf
---
Relevant document: /Users/chandu/Documents/hiring-assistant/data/AVIATION/22232367.pdf
---
Relevant document: /Users/chandu/Documents/hiring-assistant/data/ENGINEERING/30542184.pdf
---
Relevant document: /Users/chandu/Documents/hiring-assistant/data/BANKING/29839396.pdf
---
Relevant document: /Users/chandu/Documents/hiring-assistant/data/SALES/24610685.pdf
---
Relevant document: /Users/chandu/Documents/hiring-assistant/data/ENGINEERING/37335325.pdf
---
Relevant document: /Users/chandu/Documents/hiring-assistant/data/HEALTHCARE/32563518.pdf
---
Relevant document: /Users/chandu/Documents/hiring-assistant/data/DESIGNER/29524570.pdf
---
Relevant document: /Users/chandu/Documents/hiring-assistant/data/AUTOMOBILE/28790806.p

# Resume Ranking using pre-defined metrics

In [23]:
#Matching Skills and Missing Skills
matching_skills_prompt = """

System: You are an expert in human resources and you are an expert at matching skills from a job description to a CV of a candidate
Human: Please extract first the skills from the job description. 
The job description part starts with === 'JOB DESCRIPTION:' === and ends with === 'END JOB DESCRIPTION' ===.
The CV (curriculum vitae of a candidate) description part starts with === CV START: === and ends with === CV END: ===. 


The matching skills are the skills in the job description part which are also found in the CV (curriculum vitae of a candidate) description part.
The missing skills are those skills which are in the jobo description part but not in the CV (curriculum vitae of a candidate) description part.

Please extract first the job titles along with years of experience from the CV (curriculum vitae of a candidate). The extracted job titles should be related to the job description.

Then output the matching, missing in JSON structure 


with keys "matching skills" : [] , "missing skills": [], "job titles with months of experience" : dict, "educational experience": [].


Also Extract Educational Background relevant to the job description. 

Here are some examples of skills that you might find in the job descriptions and CVs:
- Wordpress
- Creating Wordpress websites
- Website Optimization
- PHP
- SQL
- Javascript
- Debugging
- HTML
- HTML5
- CSS
- CSS3
- WOO-Commerce Management
- Client Support
- Python 
- Linux, macOS, and Windows
- Git
- Building E-commerce stores using woocmmerce plugin
- Front-end development
- Codeigniter
- Programming languages: C, C++
- Machine Learning
- Deep Learning
- Database: MySQL
- Database: MongoDB
- IDEs: IntelliJ
- Azure Logic apps
- Azure Data Factor
- Azure Functions
- Experience with REST APIs
- Experience with Business Intelligence BI
- Analytical reporting using PowerBI
- Exposure to ITIL


Here is an example of how you extract matching and missing skills:

====== Example start: ======
=== 'JOB DESCRIPTION:' ===
Application Support Analyst (L2/L3) at Onepoint (PUNE_JD2023-02_IN.007)

Minimum Bachelor's Degree in Computer Science or equivalent stream.

2-3 years experience in Cloud-based (Azure, AWS and Google Cloud) Application Support,
ideally ina Managed service environment..

Experience in supporting core business applications on Azure (Logic apps, Data Factory,
Functions) and Snowflake or equivalent cloud data warehouse.

Experience with REST APls, SQL, JSON, XML.

Experience with Business Intelligence BI and Analytical reporting using PowerBI.
Knowledge in programming languages like Python, Java.

Knowledge about integration platforms or low code platforms.

Exposure to ITIL.

Knowledge in Al, ML preferred.

Excellent written and verbal communication skills in English.

Excellent interpersonal skills to collaborate with various stakeholders.

A learning enthusiast who would quickly pick up new programming languages, technologies,
and frameworks.

A proactive Self-Starter with excellent time management skills.

Problem-solving and analytical skills across technical, product, and business questions.
=== 'END JOB DESCRIPTION' ===

=== CV START: ===

Experienced web developer with a proven track record of creating dynamic and user-friendly websites. Proficient
in various programming languages and frameworks, with a strong emphasis on front-end development.
Committed to delivering high-quality code and exceptional user experiences. Strong problem-solving and
communication skills, with the ability to collaborate effectively in cross-functional teams.

Key Skills: HTML5, CSS3, JavaScript, Bootstrap, Responsive Design, UX/UI, RESTful APIs, JSON, XML, Git, Agile
Development, SEO Optimization, Performance Optimization, Cross-Browser Compatibility, Testing and
Debugging, WordPress, PHP, MySQL.

Profile Summary: Highly skilled web developer with expertise in HTML5, CSS3, and JavaScript.Focus on
building responsive and intuitive user interfaces. Strong understanding of UX/UI principles and ability to optimize
websites for performance and SEO. Experienced in working with RESTful APIs and version control systems like
Git. Collaborative team player with excellent problem-solving abilities and a passion for staying up-to-date with
the latest industry trends and technologies. Extensive experience in WordPress development and familiarity with
back-end technologies like PHP, MySQL and SQL. Effective communicator with a proven ability to work in fast-paced,
Agile environments.
=== CV END: ===

The matching skills are: RESTful APIs, SQL, JSON, XML
The missing skills are: Cloud-based Application Support, integration platforms, Business Intelligence BI, Analytical reporting using PowerBI, Exposure to ITIL, Knowledge in Al, Azure Logic apps, Azure Data Factory



=== 'JOB DESCRIPTION' ===
{job_description}
=== END JOB DESCRIPTION ===

=== CV START ===
{resume}
=== END CV ===

only give out the JSON output
"""

shortlist = []

for resume in final_list[:5]:
    loader = PyPDFLoader(resume)
    document = loader.load()

    loaded_resume = " ".join([doc.page_content for doc in document])
    llm_prompt = PromptTemplate.from_template(matching_skills_prompt).format(job_description = job_description, resume = loaded_resume)

    # print(model.invoke(llm_prompt).content)
    response = model.invoke(llm_prompt).content
    match = {}
    try: 
        match = json.loads(response[7:-3])
    except:
        # For now if there is an exception discard the example
        continue
    

    # educational_prompt = PromptTemplate.from_template(educational_prompt).format(job_description = job_description, resume = loaded_resume)
    # response = model.invoke(educational_prompt).content
    # print(match)
    
    num_matches = len(match['matching skills'])
    num_misses = len(match['missing skills']) 
    num_months = sum(match['job titles with months of experience'].values()) if len(match['job titles with months of experience'].values()) >= 1 else 0
    num_degrees = len(match['educational experience'])
    score = (num_matches * 2) - (num_misses * 1) +(num_months * 0.5) + (num_degrees * 1)

    # print(score)

    shortlist.append((score, resume))


shortlist.sort()
shortlist.reverse()

print(shortlist)



    



[(184.0, '/Users/chandu/Documents/hiring-assistant/data/ENGINEERING/28762662.pdf'), (182.0, '/Users/chandu/Documents/hiring-assistant/data/ENGINEERING/30542184.pdf'), (142.0, '/Users/chandu/Documents/hiring-assistant/data/AVIATION/22232367.pdf'), (52.5, '/Users/chandu/Documents/hiring-assistant/data/CONSULTANT/22351830.pdf'), (28.0, '/Users/chandu/Documents/hiring-assistant/data/BPO/41152404.pdf')]


# Generating Few Shot Examples

In [15]:
url = "https://www.roberthalf.com/us/en/insights/hiring-help/the-best-phone-screen-interview-questions"  # Replace with your actual URL
loader = WebBaseLoader(url)
data = loader.load()

model = ChatOpenAI(model="gpt-4o")
prompt_message = """
    Below is a webpage data of sample phone interview questions. Generate some few shot examples of the form
    Question: 
    Answer: 

    which would be provided as an example for another AI model. 
    

    ***
    {web_data}
    ***

    Just give me examples with no introduction

"""

web_data = "\n".join([doc.page_content for doc in data])

prompt = PromptTemplate.from_template(prompt_message).format(web_data = web_data)

few_shots = model.invoke(prompt).content
print(f"Examples: {few_shots}")


Examples: Question: Can you tell me about your background?
Answer: I have a Bachelor's degree in Computer Science and have been working as a software developer for the past three years. My experience includes working with Java, Python, and JavaScript, and I have contributed to several projects that focus on web development and cloud computing.

Question: Why are you looking for a new job?
Answer: I am looking for new challenges and opportunities to grow my skill set. While I have enjoyed my time at my current company, I believe that your organization offers the kind of dynamic environment where I can further develop my abilities and contribute meaningfully.

Question: How much would you like to earn in this position?
Answer: Based on my research and current market trends for this role, I am looking for a salary in the range of $70,000 to $80,000 per year. However, I am open to discussing this further and considering the complete compensation package.

Question: What attracted you to ap

# Initializing a Model for Chat Inference


In [16]:
model = ChatOpenAI(model="gpt-4o")

system_template = """You are an expert software engineering recruiter. You are given a resume from the user. Enclosed in *** is the job description for which you will be conducting an interview. The interview questions and the responses will be used by another recruiter to evaluate the candidate's fit with the company. 
So ask questions that would cover all requirements mentioned in the job requirements. 

Below are examples of good interview interactions. Use these as a guide for your style and depth of questioning:

{few_shot_examples}

Below are the steps you must perform. Ask only one question at a time. 
1) Greet the candidate based on the first name found on the resume and Read out a summarized version of the job description to the candidate. 
2) Begin the interview by discussing the educational background. Read out the job's requirement for education and mention if the candidate's education meets the demand. Follow up with a question relating to the coursework or any other academic involvement that would answer any inconsistency with the job requirement. 
Ask any follow up questions if necessary. 
3) Move on to the work experience of the candidate. For each position in the candidate's work experience, determine if that experience matches the job requirement. If there is an exact match in the technologies
ask a technical question confirming the legibility of the experience. If there is a relevant technology but not an exact match (like a different programming language experience but in the same domain), ask about how the candidate
thinks his experience aligns with the requirement. 
4) Do the same with the Projects section if the candidate has any. Ask questions only if necessary if there is any inconsistencies with the job requirement. 
5) Ask 1-2 technical questions on topics that are most important to the job. 

***{job_description}***
"""

human_template = """{resume}"""

##### Job 1: Software Engineer at Apple: https://jobs.apple.com/en-us/details/200556628/software-engineer

In [17]:
job_description = """

    Title: Software Engineer
    Summary: Be a significant part of the Hardware Test Engineering Core Infrastructure team that is at the center of the Apple Factory’s Development Process. This team is responsible for application development to improve delivery, management, and scalability of factory test platforms to factories located worldwide. We are looking for a curious, hardworking, self-motivated software developer to work on several key systems as a member of our collaborative team.
    Job Posted Date: Jun 25, 2024
    Description: We are currently seeking an individual with strong backend software skills and a willingness to work with both C++ and Swift. 
You will work with a team of extraordinary engineers to develop and deploy mass-production friendly solutions for all Apple products. This would include architecting, developing, deploying and triaging software used in the factory during manufacture of new products. Responsibilities would include triaging and diagnosing issues with cross-functional teams from diverse engineering backgrounds during new product bringup, as well as working with external vendors on defining better solutions and processes.
    Minimum Qualifications: [Backend software development experience, Extensive expertise with Objective-C, C++, or Swift,Proficient with UNIX/Linux or macOS, Experience with scripting languages (Bash, Python, Perl), Excellent knowledge of software testing methodologies & practices, Excellent knowledge of source code management concepts and systems (Git), Effective communication and an ability to collaborate with those around them]
    Preferred Qualifications: [4+ years of backend software development experience in Unix or Linux environment with expertise in C++, BS degree in Computer Science, Computer Engineering or equivalent practical experience.]
    Pay and Benefits: At Apple, base pay is one part of our total compensation package and is determined within a range. This provides the opportunity to progress as you grow and develop within a role. The base pay range for this role is between $131,500 and $243,300, and your base pay will depend on your skills, qualifications, experience, and location.

Apple employees also have the opportunity to become an Apple shareholder through participation in Apple’s discretionary employee stock programs. Apple employees are eligible for discretionary restricted stock unit awards, and can purchase Apple stock at a discount if voluntarily participating in Apple’s Employee Stock Purchase Plan. You’ll also receive benefits including: Comprehensive medical and dental coverage, retirement benefits, a range of discounted products and free services, and for formal education related to advancing your career at Apple, reimbursement for certain educational expenses — including tuition. Additionally, this role might be eligible for discretionary bonuses or commission payments as well as relocation. Learn more about Apple Benefits.

Note: Apple benefit, compensation and employee stock programs are subject to eligibility requirements and other terms of the applicable plan or program.

"""

# Loading a Resume

Needs to be changed to an upload operation that a candidate can use

In [18]:
file_path = (
    "/Users/chandu/Documents/resume/Krishna Chandu Resume.pdf"
)
loader = PyPDFLoader(file_path)
document = loader.load()

resume = " ".join([doc.page_content for doc in document])

resume = "Below is my resume in text format: " + resume

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template).format(job_description = job_description, few_shot_examples = few_shots)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [19]:
prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    MessagesPlaceholder(variable_name="history"),
    human_message_prompt
])

memory = ConversationBufferMemory(return_messages=True)

conversation = LLMChain(
    memory=memory,
    prompt=prompt,
    llm=model,
    verbose= False,
)

response = conversation.run(resume=resume)
print(response)

/Users/chandu/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/Users/chandu/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Hello Krishna,

Thank you for joining us today. Based on your resume, I see you have a strong background in computer science with both a Bachelor's and Master's degree. I'll start by summarizing the job description for the Software Engineer position you applied for at Apple.

### Job Summary:
This role involves being part of the Hardware Test Engineering Core Infrastructure team at Apple, focusing on application development to improve delivery, management, and scalability of factory test platforms. You will be working with C++ and Swift, among other technologies, to develop and deploy solutions for Apple products in mass production. Responsibilities include triaging and diagnosing issues with cross-functional teams and working with external vendors.

### Minimum Qualifications:
- Backend software development experience
- Extensive expertise with Objective-C, C++, or Swift
- Proficiency with UNIX/Linux or macOS
- Experience with scripting languages (Bash, Python, Perl)
- Excellent knowl

In [20]:
while True:
    user_input = input("Human: ")
    if user_input.lower() in ['exit', 'quit', 'bye']:
        break
    response = conversation.run(resume=user_input)
    print(response)

print("Chat ended.")

That sounds like valuable experience, Krishna. Integrating Swift Native modules for a cross-platform mobile application indeed aligns well with the requirement for expertise in Swift. Let's move on to your work experience to delve deeper into your practical exposure.

### Work Experience:
You have listed several roles that cover a range of technologies and responsibilities. Let's go through them one by one.

**1. Student Researcher at UC San Diego (June 2023 - Present):**
- Developing "VirtualPT," a mobile application using Machine Learning for remote physical therapy.
- Implementing a voice-controlled chatbot using React Native and ExpressJS.

2. **Question**: In this role, you mentioned implementing a voice-controlled chatbot. Can you explain the backend technologies you used and how you ensured robust performance and scalability for real-time video understanding?



KeyboardInterrupt: 

In [141]:
print(memory)

chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Below is my resume in text format: Krishna Vamsi Chandu\nkrishnavchandu@gmail.com |linkedin.com/in/kchand23 |www.kvchandu.com\nExperience\nStudent Researcher, Mobile Systems Design Lab June 2023 – Present\nUniversity of California, San Diego La Jolla, CA\n•Developed a commercial medically licensed mobile application in React Native , and AWS for remote physical therapy\nby utilizing 3D pose estimation and segmentation models to provide guidance and evaluation to patients.\n•Evaluated multiple state-of-the-art activity recognition models using a custom video dataset for reliable real-time activity\ndetection and repetition counting across diverse exercises. Achieved the best accuracy of 80% in counting repetitions.\n•Implemented the backend of a Voice-Controlled assistant for the application, leveraging LLMs, and Express to enhance\npatient interaction and exercise experience. Designed new UI components for seamless 

In [8]:
def format_conversation(memory):
    formatted_chat = []
    for message in memory.chat_memory.messages:
        if isinstance(message, HumanMessage):
            formatted_chat.append(f"Candidate: {message.content}")
        elif isinstance(message, AIMessage):
            formatted_chat.append(f"AI Recruiter: {message.content}")
    
    return "\n\n".join(formatted_chat)

# After the chat loop ends
formatted_conversation = format_conversation(memory)
print("Formatted Conversation:")
print(formatted_conversation)


Formatted Conversation:
Candidate: Below is my resume in text format: W I L H E L M I N A
R O B I N
A r c h i t e c t
wil.robin@email.com
(123) 456-7890
San Francisco, CA
LinkedIn
C A R E E R  S U M M A R Y
Creative and licensed architect
with 13 years of experience
looking for a collaborative
opportunity to tackle
challenging builds and stretch
my creative and technical
limits. As an architect with
Kerman Morris Architects, I
know I can provide
personalized attention and
create designs that maximize
efﬁciency and beauty.
E D U C A T I O N
Master of Architecture
University of California, SF
2007 - 2009
San Francisco, CA
Bachelor of Architecture
University of California, SF
2002 - 2006
San Francisco, CA
S K I L L S
Adobe Creative Suite
Sketchup Pro
AutoCAD
Commercial Building
Design
Residential Building Design
California Building Codes
Research
Multi-taskingW O R K  E X P E R I E N C E
Architect
EDG
2016 - current San Francisco, CA
Designed 173 blueprints for corporate clients
Supervise

In [9]:
with open("interview_transcript_2.txt", "w") as f:
    f.write(formatted_conversation)

#### 2nd LLM - Reviewing the Conversation and giving out a feedback and questionnaire 

In [10]:
def analyze_interview(conversation, job_description):
    model = ChatOpenAI(model="gpt-4")

    system_template = """You are an expert HR analyst. You will review a job description and an interview transcript, then provide:
    1. A summary report highlighting the candidate's strengths and weaknesses
    2. A detailed questionnaire for a follow-up HR interview

    The summary should include:
    - Key qualifications and how they match the job requirements
    - Technical skills assessment
    - Soft skills assessment
    - Areas where the candidate excels
    - Areas where the candidate may need improvement
    - Overall recommendation (Strongly Recommend, Recommend, Consider, or Do Not Recommend)

    The questionnaire should include:
    - 5-7 questions that probe deeper into areas where more information is needed
    - Questions should be specific and based on the candidate's responses in the initial interview
    - Include a mix of technical and behavioral questions

    Job Description:
    {job_description}

    Interview Transcript:
    {conversation}
    """

    human_template = "Please provide the summary report and questionnaire based on the above job description and interview transcript."

    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", system_template),
        ("human", human_template)
    ])

    chain = LLMChain(llm=model, prompt=chat_prompt)

    result = chain.run(job_description=job_description, conversation=conversation)

    return result


analysis_result = analyze_interview(formatted_conversation, job_description)


print("Interview Analysis and HR Questionnaire:")
print(analysis_result)



Interview Analysis and HR Questionnaire:
Summary Report:

Key Qualifications and Match to Job Requirements: 
The candidate, Wilhelmina, has an extensive background in architecture with several years of experience working as an architect. There is a significant mismatch between her qualifications and the job requirements for the Software Engineer position, as she lacks the required technical skills and experience in backend software development and programming languages such as C++, Swift, or Objective-C.

Technical Skills Assessment: 
Wilhelmina lacks the necessary technical skills for the Software Engineer position. She has not demonstrated any experience or knowledge in backend software development, programming languages, UNIX/Linux or macOS, or other key technical requirements for the job.

Soft Skills Assessment: 
Wilhelmina has shown good communication skills and honesty during the interview. She also seems to have demonstrated leadership skills in her previous role, as she manage

In [11]:
with open("interview_analysis_2.txt", "w") as f:
    f.write(analysis_result)